In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
import torch
import pandas as pd

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
# outputs = model(**inputs, labels=labels)
# loss = outputs.loss
# logits = outputs.logits

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [3]:
dataset = load_dataset('json', data_files={'train': 'train.jsonl', 'test': 'test.jsonl'})
print(dataset)


Using custom data configuration default-5a3e3e29c398a4dc
Reusing dataset json (C:\Users\kjp19\.cache\huggingface\datasets\json\default-5a3e3e29c398a4dc\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['word', 'sentence1', 'sentence2', 'idx', 'label', 'start1', 'start2', 'end1', 'end2', 'version'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['word', 'sentence1', 'sentence2', 'idx', 'label', 'start1', 'start2', 'end1', 'end2', 'version'],
        num_rows: 638
    })
})


In [41]:
#Pre process Data for training here:
encoded_dataset = dataset.map(lambda examples: tokenizer(examples['sentence1']), batched=True)

encoded_dataset.save_to_disk("encoded_dataset")


train_dataset = encoded_dataset['train']
test_dataset = encoded_dataset['test']

Loading cached processed dataset at C:\Users\kjp19\.cache\huggingface\datasets\json\default-5a3e3e29c398a4dc\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426\cache-304a5a5571ead234.arrow
Loading cached processed dataset at C:\Users\kjp19\.cache\huggingface\datasets\json\default-5a3e3e29c398a4dc\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426\cache-e192044695208203.arrow


In [42]:
training_args = TrainingArguments("test_trainer")
trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=test_dataset)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: start2, idx, end1, word, end2, sentence1, start1, version, sentence2.
***** Running training *****
  Num examples = 5000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1875


ValueError: expected sequence of length 10 at dim 1 (got 6)